In [3]:
import pandas as pd
import numpy as np

### Сводная таблица №1:
Все потоки на станциях Рижского направления с учетом подтипов билетов, признака наличия льгот и типов поездов.


In [129]:
data = pd.read_excel('/Users/eugenia/Downloads/2019 ноя-дек.xlsb', engine = 'pyxlsb')

In [130]:
data.head()

,O2. Кол-во отправленных пассажиров,Отчетный год,Отчетный месяц,Группа льгот ЦО-22 РЖД,Размер скидки текстом,Профиль льготы,Рынок,Полигон,Код станции отправления,Наименование станции отправления,Регион станции отправления,Код станции назначения,Наименование станции назначения,Регион станции назначения,Подтип билета,Исходный билет,"Признак ""Туда и обратно""",Категория поезда
0,1.0,2019,ноябрь,Вне набора социальн.услуг,100% СКИДКИ,ВОХР ЖД РФ,БМО,МТУ,2000007,МОСКВА КИЕВСКАЯ,г. Москва,2000471,БЕКАСОВО-СОРТ.,г. Москва,Разовый,Пригородный ЖД билет,Нет,Пригородный пассажирский поезд
1,1.0,2019,ноябрь,Вне набора социальн.услуг,100% СКИДКИ,ВОХР ЖД РФ,БМО,МТУ,2000082,КУБИНКА 1,Московская область,2000471,БЕКАСОВО-СОРТ.,г. Москва,Разовый,Пригородный ЖД билет,Нет,Пригородный пассажирский поезд
2,1.0,2019,ноябрь,Вне набора социальн.услуг,100% СКИДКИ,ВОХР ЖД РФ,БМО,МТУ,2000612,ИВАНЦЕВО,Московская область,2001077,ИКША,Московская область,Разовый,Пригородный ЖД билет,Да,Пригородный пассажирский поезд
3,5.0,2019,ноябрь,Вне набора социальн.услуг,100% СКИДКИ,ВОХР ЖД РФ,БМО,МТУ,2000912,ОРЕХОВО-ЗУЕВО,Московская область,2000920,АЛЕКСАНДРОВ 1,Владимирская область,Разовый,Пригородный ЖД билет,Нет,Пригородный пассажирский поезд
4,1.0,2019,ноябрь,Вне набора социальн.услуг,100% СКИДКИ,ВОХР ЖД РФ,БМО,МТУ,2000920,АЛЕКСАНДРОВ 1,Владимирская область,2000912,ОРЕХОВО-ЗУЕВО,Московская область,Разовый,Пригородный ЖД билет,Нет,Пригородный пассажирский поезд


In [186]:
corrs = pd.read_excel('fin_corrs.xlsx')

In [109]:
corrs.head()

,source,target,year
0,МОСКВА КУРСКАЯ,МОСКВА-КАЛАНЧЕВСКАЯ,2019
1,МОСКВА КУРСКАЯ,РИЖСКАЯ (РЖЕВСКАЯ),2019
2,МОСКВА КУРСКАЯ,МОСКВА РИЖСКАЯ,2019
3,МОСКВА КУРСКАЯ,ДМИТРОВСКАЯ,2019
4,МОСКВА КУРСКАЯ,ГРАЖДАНСКАЯ,2019


In [119]:
df = pd.DataFrame(columns = ['source', 'target', 'year', 'month', 'ticket', 'benefits', 'train'])
for i in range(1,12):
    data = pd.read_excel(f"/Users/eugenia/Данные ЦППК/{i}.xlsb", engine = 'pyxlsb')
    data = data[['Отчетный год', 'Отчетный месяц', 'Наименование станции отправления', 'Наименование станции назначения', 'O2. Кол-во отправленных пассажиров', 'Группа льгот ЦО-22 РЖД', 'Категория поезда', 'Подтип билета']]
    data['benefits'] = np.where(data['Группа льгот ЦО-22 РЖД']=='ПЛАТНЫЕ', 0, 1)
    data['train'] = np.where(data['Категория поезда'] == 'Пригородный пассажирский поезд', 0, 1)
    data.columns = ['year', 'month', 'source', 'target', 'flows', 'ben_type', 'train_cat', 'ticket', 'benefits', 'train']
    data = data[(data['source'].isin(stations)) & (data['target'].isin(stations))]
    fin = data.groupby(['source', 'target', 'year', 'month', 'ticket', 'benefits', 'train'], as_index=False)['flows'].sum()
    df = pd.concat([df,fin])
    #corrs = corrs.merge(fin, on=['source', 'target', 'year', 'month'], how = 'left')

In [124]:
df = df.sort_values(by = ['source', 'target', 'year'])
df.to_excel('table1.xlsx')

### Сводная таблица №2:
Потоки на станциях: 1) в рамках Рижского направления 2) транзитные потоки

In [10]:
data = pd.read_csv('df_all.csv') # все потоки в разрезе год-месяц для всех ж/д станций (с учетом льгот)

In [ ]:
month_data = data.groupby(['source','target','year','month'], as_index=False)['flows'].sum() # группировка по месяцам

In [4]:
corrs = pd.read_excel('fin_corrs.xlsx')
corrs.head() 

,source,year,month
0,МОСКВА КУРСКАЯ,2019,январь
1,МОСКВА КУРСКАЯ,2019,февраль
2,МОСКВА КУРСКАЯ,2019,март
3,МОСКВА КУРСКАЯ,2019,апрель
4,МОСКВА КУРСКАЯ,2019,май


In [16]:
stations = list(corrs.source.drop_duplicates())
len(stations) # все станции рижского направления

47

In [162]:
list_rizhskoe = []
list_transit = []
for i, r in corrs.iterrows():
    sub_data = month_data[(month_data['source'] == r['source']) &
                          (month_data['year'] == r['year']) & 
                          (month_data['month'] == r['month'])]
    new_list = [a for a in stations if a not in r['source']]
    stations_data = sub_data[sub_data['target'].isin(new_list)] # фильтрация по списку станций - Рижское 
    transit_data = sub_data[~sub_data['target'].isin(stations)] # все транзитные потоки
    list_rizhskoe.append(stations_data.flows.sum())
    list_transit.append(transit_data.flows.sum())

In [163]:
corrs['stations'] = list_rizhskoe
corrs['transit'] = list_transit
corrs.to_excel('table2.xlsx')